In [ ]:
import os
import pandas as pd
from datetime import datetime

# Path to the folder containing Excel files
folder_path = '/Users/soumilhooda/Desktop/Data-WD/Renewables-Messy'

# Path to the folder where CSV files will be saved
output_folder = '/Users/soumilhooda/Desktop/Data-WD/Renewables-Clean'

# Define a function to extract English version after '/'
def extract_english_version(state):
    parts = state.split('/')
    if len(parts) > 1:
        return parts[1].strip()  # Return the English version, stripped of leading/trailing whitespace
    else:
        return state.strip()  # Return the original state if no '/' found
    
# Function to process each Excel file
def process_excel_file(file_path):
    try:
        # Read Excel file with skiprows set to 4
        df = pd.read_excel(file_path, skiprows=4)

        # Extract year from the date in the third column
        year = pd.to_datetime(df.columns[2]).year

        # Extract relevant data
        relevant_data = df.iloc[1:42, 1:6]

        # Rename columns
        relevant_data.columns = ['State', 'Wind Energy', 'Solar Energy', 'RES', 'Total']

        # Apply the function to the 'State' column
        relevant_data['State'] = relevant_data['State'].apply(extract_english_version)

        # Generate output file name (YYYY-MM-DD.csv)
        date_str = df.columns[2]  # Assuming df.columns[2] contains the date string like "12 Jan 2022"
        output_file_name = os.path.join(output_folder, f"{year:04d}-{pd.to_datetime(date_str).strftime('%m-%d')}.csv")

        # Save relevant data to CSV
        relevant_data.to_csv(output_file_name, index=False)

        # print(f"Successfully rewrote {file_path} to {output_file_name}")
    except Exception as e:
        print(f"Failed to process {file_path}. Error: {str(e)}")

# Iterate over each file in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.xlsx') or file_name.endswith('.xls'):
        file_path = os.path.join(folder_path, file_name)
        process_excel_file(file_path)

In [ ]:
import os
import pandas as pd

# Path to the folder containing input CSV files
input_folder = '/Users/soumilhooda/Desktop/Data-WD/Renewables/Renewables-Clean'

# Path to the folder where output CSV files will be saved
output_folder = '/Users/soumilhooda/Desktop/Data-WD/Renewables/Renewables-FormWise'

# List of Indian states
indian_states = [
    'Chandigarh', 'Delhi', 'Haryana', 'Himachal Pradesh', 'Jammu & Kashmir', 'Ladakh', 'Punjab', 'Rajasthan',
    'Uttar Pradesh', 'Uttarakhand', 'Northern Region', 'Chhattisgarh', 'Gujarat', 'Madhya Pradesh', 'Maharashtra',
    'Daman & Diu', 'Dadra & Nagar Haveli', 'Goa', 'Western Region', 'Andhra Pradesh', 'Telangana', 'Karnataka',
    'Kerala', 'Tamil Nadu', 'Puducherry', 'Southern Region', 'Bihar', 'Jharkhand', 'Odisha', 'West Bengal',
    'Sikkim', 'Eastern Region', 'Arunachal Pradesh', 'Assam', 'Manipur', 'Meghalaya', 'Mizoram', 'Nagaland',
    'Tripura', 'North-Eastern Region', 'All India'
]

# Create empty DataFrames for Wind Energy, Solar Energy, RES, and Total Renewable Energy
wind_energy_df = pd.DataFrame(index=pd.date_range(start='2021-01-01', end='2023-12-31'), columns=indian_states)
solar_energy_df = pd.DataFrame(index=pd.date_range(start='2021-01-01', end='2023-12-31'), columns=indian_states)
res_df = pd.DataFrame(index=pd.date_range(start='2021-01-01', end='2023-12-31'), columns=indian_states)
total_renewable_energy_df = pd.DataFrame(index=pd.date_range(start='2021-01-01', end='2023-12-31'), columns=indian_states)

# Iterate through each input CSV file
for file_name in os.listdir(input_folder):
    if file_name.endswith('.csv'):
        file_path = os.path.join(input_folder, file_name)
        df = pd.read_csv(file_path)
        
        # Extract date from the file name
        date_str = file_name.split('.')[0]

        # Iterate through each state and fill the respective DataFrame
        for state in indian_states:
            state_data = df[df['State'] == state]
            if not state_data.empty:
                wind_energy_df.loc[date_str, state] = state_data['Wind Energy'].values[0]
                solar_energy_df.loc[date_str, state] = state_data['Solar Energy'].values[0]
                res_df.loc[date_str, state] = state_data['RES'].values[0]
                total_renewable_energy_df.loc[date_str, state] = state_data['Total'].values[0]

        # Print progress
        print(f"Processed data for file: {file_name}")

# Save DataFrames to separate output CSV files
wind_energy_df.to_csv(os.path.join(output_folder, 'WindEnergy.csv'))
solar_energy_df.to_csv(os.path.join(output_folder, 'SolarEnergy.csv'))
res_df.to_csv(os.path.join(output_folder, 'RES.csv'))
total_renewable_energy_df.to_csv(os.path.join(output_folder, 'Total.csv'))

In [ ]:
import os
import pandas as pd

# Path to the folder containing power stocks CSV files
folder_path = '/Users/soumilhooda/Desktop/WD/Data-WD/Energy Stocks'

# List to store dataframes for each stock
stock_dfs = []

# Iterate through each CSV file in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        stock_name = file_name.split('.')[0]  # Extract stock name from the filename
        
        try:
            # Read CSV file
            df = pd.read_csv(file_path)
            df['Date'] = pd.to_datetime(df['Date'])  # Convert 'Date' column to datetime
            df.set_index('Date', inplace=True)  # Set 'Date' column as index
            
            # Calculate average of 'High' and 'Low' columns
            df[f'AVG_{stock_name}'] = (df['High'] + df['Low']) / 2
            
            # Rename 'Volume' column to include stock name
            df.rename(columns={'Volume': f'Volume_{stock_name}'}, inplace=True)
            
            # Add dataframe to the list
            stock_dfs.append(df[[f'AVG_{stock_name}', f'Volume_{stock_name}']])
            
            # Print progress
            print(f"Processed data for file: {file_name}")
        except Exception as e:
            print(f"Failed to process data for file: {file_name}, Error: {e}")

# Create a dataframe indexed by dates from 01-01-1997 to 31-12-2023
date_range = pd.date_range(start='1997-01-01', end='2023-12-31')
overall_df = pd.DataFrame(index=date_range)

# Merge dataframes for all stocks
for df in stock_dfs:
    overall_df = overall_df.merge(df, how='left', left_index=True, right_index=True)

# Fill NULL values with 'NULL'
overall_df.fillna('NULL', inplace=True)

# Save merged dataframe to CSV file
overall_df.to_csv(os.path.join(folder_path, 'Overall_PowerStocks.csv'))


In [ ]:
# Read Electricity1 and Electricity2 CSV files
electricity1 = pd.read_csv('/Users/soumilhooda/Desktop/WD/Data-WD/Electricity/Electricity1.csv')
electricity2 = pd.read_csv('/Users/soumilhooda/Desktop/WD/Data-WD/Electricity/Electricity2.csv')

# Convert date columns to consistent format (YYYY-MM-DD)
electricity1['Date'] = pd.to_datetime(electricity1['Date'], format='%d-%m-%Y')
electricity2['Date'] = pd.to_datetime(electricity2['Date'], format='%m/%d/%y')

# Fill 0 values with NULL
electricity1 = electricity1.replace(0, pd.NA)
electricity2 = electricity2.replace(0, pd.NA)

# Concatenate the two dataframes
concatenated_df = pd.concat([electricity1, electricity2])

# Sort by dates
concatenated_df.sort_values(by='Date', inplace=True)

# Save the result to a new CSV file
concatenated_df.to_csv('electricity.csv', index=False)


In [ ]:
import pandas as pd

# Read the Excel file
mcx = pd.read_excel('/Users/soumilhooda/Desktop/WD/Data-WD/MCX-Commodity-Trading-Statistics/Messy/trading-statistics-apr-2016.xlsx', skiprows=4)

# Rename columns for clarity
mcx.columns = ['Year', 'Month', 'Date', 'Useless1', 'Equity Type', 'Commodity Name', 'Useless2', 'Expiry', 'Price Unit', 'C/P Option', 'Strike Price', 'Open Price', 'Useless3', 'Useless4', 'Useless5', 'Useless6', 'Spot Price', 'Useless7', 'Useless8', 'Trading Volume Lots', 'Useless9', 'Lot Size Units', 'Useless10', 'Useless11', 'Useless12', 'Useless13']

# Drop columns containing 'Useless' in their name
mcx = mcx.drop(columns=mcx.columns[mcx.columns.str.contains('Useless')])

In [ ]:
import os
import pandas as pd
from datetime import datetime, timedelta

# Function to calculate the last day of the month
def last_day_of_month(year, month):
    if month == 2:  # February
        if year % 4 == 0 and (year % 100 != 0 or year % 400 == 0):  # Leap year check
            return 29  # Leap year
        else:
            return 28  # Non-leap year
    elif month in [4, 6, 9, 11]:  # Months with 30 days
        return 30
    else:
        return 31  # All other months

# Function to process each file
def process_file(input_path, output_folder):
    try:
        # Read the Excel file
        mcx = pd.read_excel(input_path, skiprows=4)
        
        # Rename columns for clarity
        mcx.columns = ['Year', 'Month', 'Date', 'Useless1', 'Equity Type', 'Commodity Name', 'Useless2', 'Expiry', 'Price Unit', 'C/P Option', 'Strike Price', 'Open Price', 'Useless3', 'Useless4', 'Useless5', 'Useless6', 'Spot Price', 'Useless7', 'Useless8', 'Trading Volume Lots', 'Useless9', 'Lot Size Units', 'Useless10', 'Useless11', 'Useless12', 'Useless13']
        
        # Drop columns containing 'Useless' in their name
        mcx = mcx.drop(columns=mcx.columns[mcx.columns.str.contains('Useless')])
        
        # Filter rows based on commodity name
        mcx = mcx[mcx['Commodity Name'].isin(['CRUDEOIL', 'NATURALGAS', 'COTTON', 'ENRGDEX'])]
        
        # Save the modified file to the output folder
        output_filename = os.path.join(output_folder, os.path.splitext(os.path.basename(input_path))[0] + ".csv")
        mcx.to_csv(output_filename, index=False)
        
        # print(f"Processed: {input_path}")
    
    except Exception as e:
        print(f"Error processing {input_path}: {e}")

# Input folder containing the files
input_folder = '/Users/soumilhooda/Desktop/WD/Data-WD/MCX-Commodity-Trading-Statistics/Messy'

# Output folder to save modified files
output_folder = '/Users/soumilhooda/Desktop/WD/Data-WD/MCX-Commodity-Trading-Statistics/Cleaned'

# Iterate through all files in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.xlsx'):
        input_path = os.path.join(input_folder, filename)
        process_file(input_path, output_folder)

print("All files processed.")

In [ ]:
import pandas as pd
agri = pd.read_csv("/Users/soumilhooda/Desktop/WD/Data-WD/Agriculture-Water/Agriculture-Water.csv")

In [ ]:
agri

In [ ]:
import csv
from collections import defaultdict

def process_data(data_filename, weights_filename):
    """
    Processes agricultural data, replaces missing data with 0, and calculates 
    weighted average rainfall requirements.

    Args:
        data_filename (str): Path to the CSV file with agricultural data.
        weights_filename (str): Path to the CSV file with crop weights.

    Returns:
        dict: Dictionary with keys as (state, month_number) and values as 
              the weighted average rainfall requirement for that state and month.
    """

    rainfall_requirements = defaultdict(lambda: (0.0, 0.0))  # (total_rainfall, total_weight)

    # Load crop weights from the weights file
    crop_weights = {}
    with open(weights_filename, 'r') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # Skip header
        for row in reader:
            crop_name, weight = row
            crop_weights[crop_name.strip()] = float(weight)

    # Process agricultural data
    with open(data_filename, 'r') as csvfile:
        reader = csv.reader(csvfile)
        header = next(reader)

        # Extract state names and crop names from headers
        states = []
        crops = []
        for col_header in header[1:]:
            if '/' in col_header:
                state, crop = col_header.split('/')
                states.append(state.strip())
                crops.append(crop.strip())
            else:
                states.append(None)
                crops.append(None)

        for row in reader:
            date, *crop_data = row
            month_number = int(date.split('-')[1])

            for i, (state, crop) in enumerate(zip(states, crops)):
                if state and crop:
                    crop_value = crop_data[i] or 0.0  # Replace missing data with 0
                    weight = crop_weights.get(crop, 0.0)  # Get weight (0 if not found)

                    # Update total rainfall and weight for the state and month
                    total_rainfall, total_weight = rainfall_requirements[(state, month_number)]
                    rainfall_requirements[(state, month_number)] = (
                        total_rainfall + float(crop_value) * weight, 
                        total_weight + weight
                    )

    # Calculate weighted averages
    for key, (total_rainfall, total_weight) in rainfall_requirements.items():
        if total_weight > 0:
            rainfall_requirements[key] = total_rainfall / total_weight
        else:
            rainfall_requirements[key] = 0.0  # Handle cases with no valid crop data

    return rainfall_requirements

data_filename = "/Users/soumilhooda/Desktop/WD/Data-WD/Agriculture-Water/Agriculture-Water.csv"
weights_filename = "/Users/soumilhooda/Desktop/WD/Data-WD/Agriculture-Water/Agriculture-Weights.csv"
rainfall_dict = process_data(data_filename, weights_filename)


In [ ]:
agriculture_water = pd.read_csv("/Users/soumilhooda/Desktop/WD/Data-WD/Agriculture-Water/Agriculture-Water.csv")
agriculture_weights = pd.read_csv("/Users/soumilhooda/Desktop/WD/Data-WD/Agriculture-Water/Agriculture-Weights.csv")

In [ ]:
import pandas as pd

def calculate_weighted_monthly_rainfall(df_water, df_weights):
    """
    Calculates weighted monthly cumulative rainfall for each state and crop.

    Args:
        df_water: Dataframe containing daily rainfall data for each state and crop.
        df_weights: Dataframe containing weights for each state and crop.

    Returns:
        A dictionary with keys as (state name, month number) and values as weighted monthly rainfall.
    """

    rref = {}
    for state_crop in df_water.columns[1:]:  # Skip the 'Date' column
        state, crop = state_crop.split("/")
        weight = df_weights[state_crop].iloc[0]  # Get weight for the state-crop pair

        monthly_data = df_water[["Date", state_crop]].set_index("Date")
        monthly_data.index = pd.to_datetime(monthly_data.index)
        monthly_data["Month"] = monthly_data.index.month  # Extract month number

        monthly_rainfall = monthly_data.groupby("Month")[state_crop].sum() * 1

        for month, rainfall in monthly_rainfall.items():
            key = (state.upper(), month)
            if key not in rref:
                rref[key] = rainfall
            else:
                rref[key] += rainfall

    return rref

# Example usage:
rref = calculate_weighted_monthly_rainfall(agriculture_water, agriculture_weights)


In [ ]:
rref